## [DACON] 진동데이터 활용 충돌체 탐지 AI 경진대회
## 1Gb (팀명) (Team name)
## 2020년 월 일 (제출날짜) (Submission date)

## 1. 라이브러리 및 데이터
## Library & Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import GroupKFold
from scipy.signal import find_peaks

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import models

import sys
sys.path.append('../src')
from losses import *

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)

In [ ]:
GBS = 375
NNBATCHSIZE = 256
SPLIT = 5
LR = 0.0015
EPOCHS = 600
SEED = 62

outdir = 'wavenet_models'

if not os.path.exists(outdir):
    os.makedirs(outdir)
seed_everything(SEED)

## 2. 데이터 전처리
## Data Cleansing & Pre-Processing  

In [ ]:
train = pd.read_csv('./data/KAERI_dataset/train_features.csv')
test = pd.read_csv('./data/KAERI_dataset/test_features.csv')
train_y = pd.read_csv('./data/KAERI_dataset/train_target.csv')

train = train.merge(train_y, on='id', how='left')
del train_y

In [ ]:
def batching(df, batch_size):
    df['group'] = df.groupby(df.index//batch_size, sort=False)['S1'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df
# rolling and aggreagate batch features
def rolling_features(train, test):
    
    pre_train = train.copy()
    pre_test = test.copy()
    
        
    for df in [pre_train, pre_test]:
        
        for window in [50]:
            
            # roll backwards
            df['S1mean_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).mean())
            df['S1mean_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).mean())

            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).std())
            df['S1std_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).std())

            df['S1var_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).var())
            df['S1var_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).var())

            df['S1min_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).min())
            df['S1min_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).min())
            
            
            df['S1max_t' + str(window) + '_S1'] = df.groupby(['id'])['S1'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S2'] = df.groupby(['id'])['S2'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S3'] = df.groupby(['id'])['S3'].transform(lambda x: x.shift(1).rolling(window).max())
            df['S1max_t' + str(window) + '_S4'] = df.groupby(['id'])['S4'].transform(lambda x: x.shift(1).rolling(window).max())

            min_max = (df['S1'] - df['S1min_t' + str(window) + '_S1']) / (df['S1max_t' + str(window) + '_S1'] - df['S1min_t' + str(window) + '_S1'])
            df['norm_t' + str(window) + '_S1'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S1']) - np.ceil(df['S1min_t' + str(window) + '_S1']))
            min_max = (df['S2'] - df['S1min_t' + str(window) + '_S2']) / (df['S1max_t' + str(window) + '_S2'] - df['S1min_t' + str(window) + '_S2'])
            df['norm_t' + str(window) + '_S2'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S2']) - np.ceil(df['S1min_t' + str(window) + '_S2']))
            min_max = (df['S3'] - df['S1min_t' + str(window) + '_S3']) / (df['S1max_t' + str(window) + '_S3'] - df['S1min_t' + str(window) + '_S3'])
            df['norm_t' + str(window) + '_S3'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S3']) - np.ceil(df['S1min_t' + str(window) + '_S3']))
            min_max = (df['S4'] - df['S1min_t' + str(window) + '_S4']) / (df['S1max_t' + str(window) + '_S4'] - df['S1min_t' + str(window) + '_S4'])
            df['norm_t' + str(window) + '_S4'] = min_max * (np.floor(df['S1max_t' + str(window) + '_S4']) - np.ceil(df['S1min_t' + str(window) + '_S4']))

    del train, test, min_max
    
    
    return pre_train.fillna(0), pre_test.fillna(0)
# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        train_input_mean = train[col].mean()
        train_input_sigma = train[col].std()
        train[col] = (train[col] - train_input_mean) / train_input_sigma
        test[col] = (test[col] - train_input_mean) / train_input_sigma
    
    return train, test

# get lead and lags features
def lag_with_pct_change(df, windows):
    
    signal_col = [f'S{x}' for x in range(1, 5)]
    for col in signal_col:
        for window in windows:
            df[col + '_pos_' + str(window)] = df.groupby('group')[col].shift(window).fillna(0)
            df[col + '_shift_neg_' + str(window)] = df.groupby('group')[col].shift(-1 * window).fillna(0)
    
    return df

# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size = batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2])
    

    return df

# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [col for col in train.columns if col not in ['index', 'id', 'group', 'X', 'Y', 'M', 'V', 'S1', 'S2', 'S3', 'S4']]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]], axis = 0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features

def split(train, test, GROUP_BATCH_SIZE=4000, SPLITS=5):
    print('Reading Data Started...')
    train, test = normalize(train, test)
    train, test =  rolling_features(train, test)
    print('Reading and Normalizing Data Completed')
    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size=GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size=GROUP_BATCH_SIZE)
    
    
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')

    target = ['X', 'Y', 'M', 'V']
    group = train['group']
    kf = GroupKFold(n_splits=SPLITS)
    splits = [x for x in kf.split(train, train[target], group)]
    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])
        new_splits.append(new_split)

    train_tr = np.array(list(train.groupby('group').apply(lambda x: x[target].values))).astype(np.float32)
    train = np.array(list(train.groupby('group').apply(lambda x: x[features].values)))
    
    
    test = np.array(list(test.groupby('group').apply(lambda x: x[features].values)))
    print(train.shape, test.shape, train_tr.shape)
    return train, test, train_tr, new_splits

In [ ]:
train, test, train_tr, new_splits = split(train, test, GROUP_BATCH_SIZE=GBS, SPLITS=SPLIT)

In [ ]:
train = train[:, :200, :]
test = test[:, :200, :]

In [ ]:
train_tr = train_tr[:, 0, :]

In [ ]:
class IronDataset(Dataset):
    def __init__(self, data, labels, training=True, transform=False):
        
        self.data = data
        self.labels = labels
        
        self.training = training

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        data = self.data[idx]

        labels = self.labels[idx]

        return [data.astype(np.float32), labels.astype(np.float32)]

## 4. 변수 선택 및 모델 구축
## Feature Engineering & Initial Modeling  

In [ ]:
def kaeri_metric(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: KAERI metric
    '''
    
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)


### E1과 E2는 아래에 정의됨 ###

def E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)


def E2(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    
    
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))


def E2M(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2], np.array(y_pred)[:,2]
    
    
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

def E2V(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,-1], np.array(y_pred)[:,-1]
    
    
    return np.mean(np.square((_t - _p) / (_t + 1e-06)))

def torch_kaeri_metric(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: KAERI metric
    '''
    
    return 0.5 * torch_E1(y_true, y_pred) + 0.5 * torch_E2(y_true, y_pred)


### E1과 E2는 아래에 정의됨 ###

def torch_E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    return: distance error normalized with 2e+04
    '''
    _t, _p = y_true[:,:2], y_pred[:,:2]
    return torch.mean((_t - _p)**2 / 2e+04)
def torch_E2(y_true, y_pred, ):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    return: sum of mass and velocity's mean squared percentage error
    '''
    _t, _p = y_true[:,2:], y_pred[:,2:]
    return torch.mean(((_t - _p) / (_t + 1e-06))**2)

In [ ]:
class Wave_Block(nn.Module):
    
    def __init__(self,in_channels,out_channels,dilation_rates):
        super(Wave_Block,self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()
        
        self.convs.append(nn.Conv1d(in_channels,out_channels,kernel_size=1))
        dilation_rates = [2**i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.gate_convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=3,padding=dilation_rate,dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels,out_channels,kernel_size=1))
            
    def forward(self,x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x))*torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i+1](x)
            res = torch.add(res, x)
        return res


class Classifier_wave(nn.Module):
    def __init__(self):
        super().__init__()
        
        # For normal input
        self.wave_block1 = Wave_Block(38,64,12)
        self.bn_1 = nn.BatchNorm1d(64)

        self.wave_block2 = Wave_Block(64,64,8)
        self.bn_2 = nn.BatchNorm1d(64)

        self.wave_block3 = Wave_Block(64,64,4)
        self.bn_3 = nn.BatchNorm1d(64)

        self.wave_block4 = Wave_Block(64,128,1)
        self.bn_4 = nn.BatchNorm1d(128)
        
        self.fc = nn.Linear(128, 4)
        
        

    def flip(self, x, dim):
        dim = x.dim() + dim if dim < 0 else dim
        return x[tuple(slice(None, None) if i != dim
                 else torch.arange(x.size(i)-1, -1, -1).long()
                 for i in range(x.dim()))]
    
    def forward(self,x):
        x = x.permute(0, 2, 1)
        # forward input
        x = self.wave_block1(x)
        x = self.bn_1(x)
        x = self.wave_block2(x)
        x = self.bn_2(x)
        x = self.wave_block3(x)
        x = self.bn_3(x)
        x = self.wave_block4(x)
        x = self.bn_4(x)
#         x = x.permute(0, 2, 1)
        x = x.mean(dim=2)
        x = self.fc(x)

        return x

class Classifier_res(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv_1 = nn.Conv2d(1, 3, (3, 1))
        model = models.resnet18(pretrained=True)
        model = list(model.children())[:-1]
        model.append(nn.Conv2d(512, 4, 1))
        self.net = nn.Sequential(*model)
        
    def forward(self,x):
        
        # CNN model
        x = self.conv_1(x)
        x = self.net(x)
        x = x.squeeze(-1).squeeze(-1)

        return x
    

class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize


    def load_best_weights(self, model):
        model.load_state_dict(torch.load(self.checkpoint_path))

    def __call__(self, score, model):
        if self.best_score is None or \
                (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
            return 1
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return 2
        return 0


## 5. 모델 학습 및 검증
## Model Tuning & Evaluation

In [ ]:
def do_train(train, test, do_wave):
    
    if not do_wave:
        train = np.expand_dims(train, 1)
        test = np.expand_dims(test, 1)
        
    test_y = np.zeros([test.shape[0], 4])
    test_dataset = IronDataset(test, test_y)
    test_dataloader = DataLoader(test_dataset, NNBATCHSIZE, shuffle=False, num_workers=8, pin_memory=True)
    test_preds_all = np.zeros([test.shape[0], 4])


    oof_score = []
    for index, (train_index, val_index, _) in enumerate(new_splits[0:], start=0):
        print("Fold : {}".format(index))
        train_dataset = IronDataset(train[train_index], train_tr[train_index])
        train_dataloader = DataLoader(train_dataset, NNBATCHSIZE, shuffle=True, num_workers=6, pin_memory=True)

        valid_dataset = IronDataset(train[val_index], train_tr[val_index])
        valid_dataloader = DataLoader(valid_dataset, NNBATCHSIZE, shuffle=False, num_workers=4, pin_memory=True)

        it = 0
        if do_wave:
            model = Classifier_wave()
        else:
            model = Classifier_res()
            
        model = model.cuda()

        early_stopping = EarlyStopping(patience=200, is_maximize=False,
                                       checkpoint_path=os.path.join(outdir, "cnn_fold_{}_iter_{}.pt".format(index, it)))

        weight = None
        opt = torch.optim.Adam(model.parameters(), lr=LR)
        schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=5, factor=0.9)

        avg_train_losses, avg_valid_losses = [], []


        for epoch in range(EPOCHS):
            print('**********************************')
            print("Folder : {} Epoch : {}".format(index, epoch))
            print("Curr learning_rate: {:0.9f}".format(opt.param_groups[0]['lr']))
            train_losses, valid_losses = [], []
            tr_loss_cls_item, val_loss_cls_item = [], []


            model.train()  # prep model for training
            train_preds, train_true = torch.Tensor([]).cuda(), torch.Tensor([]).cuda()

            for (x, y) in train_dataloader:
                x = x.cuda()
                y = y.cuda()

                opt.zero_grad()


                predictions = model(x)

                loss = torch_kaeri_metric(y, predictions)

                # backward pass: compute gradient of the loss with respect to model parameters
                loss.backward()
                opt.step()
                train_losses.append(loss.item())
                train_true = torch.cat([train_true, y], 0)
                train_preds = torch.cat([train_preds, predictions], 0)


            model.eval()  # prep model for evaluation
            val_preds, val_true = torch.Tensor([]).cuda(), torch.Tensor([]).cuda()
            print('EVALUATION')
            with torch.no_grad():
                for (x, y) in valid_dataloader:
                    x = x.cuda()
                    y = y.cuda()

                    predictions = model(x)

                    loss = torch_kaeri_metric(y, predictions)
                    valid_losses.append(loss.item())

                    val_true = torch.cat([val_true, y], 0)
                    val_preds = torch.cat([val_preds, predictions], 0)

            # calculate average loss over an epoch
            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)
            avg_train_losses.append(train_loss)
            avg_valid_losses.append(valid_loss)

            if epoch % 5 ==0:
                print("train_loss: {:0.6f}, valid_loss: {:0.6f}".format(train_loss, valid_loss))

            train_score = kaeri_metric(train_true.cpu().detach().numpy(), train_preds.cpu().detach().numpy())

            val_score = kaeri_metric(val_true.cpu().detach().numpy(), val_preds.cpu().detach().numpy())

            schedular.step(valid_loss) 
            if epoch % 5 ==0:
                print("train_metric: {:0.6f}, valid_metric: {:0.6f}".format(train_score, val_score))
            res = early_stopping(valid_loss, model) #  - val_score
            if  res == 2:
                print("Early Stopping")
                print('folder %d global best val max metric model score %f' % (index, early_stopping.best_score))
                break
            elif res == 1:
                print('save folder %d global val max metric model score %f loss %f' % (index, val_score, valid_loss))
        print('Folder {} finally best global max metric score is {}'.format(index, early_stopping.best_score))
        oof_score.append(round(early_stopping.best_score, 6))

        model.load_state_dict(torch.load(os.path.join(outdir, "cnn_fold_{}_iter_{}.pt".format(index, it))))
        model.eval()
        pred_list = []
        with torch.no_grad():
            for x, y in tqdm(test_dataloader):
                x = x.cuda()
                y = y.cuda()

                predictions = model(x)
                pred_list.append(predictions.cpu().numpy())
            test_preds = np.vstack(pred_list)
            test_preds_all += test_preds / SPLIT
    print('all folder score is:%s'%str(oof_score))
    print('OOF mean score is: %f'% (sum(oof_score)/len(oof_score)))
    print('Generate submission.............')
    submission_csv_path = './data/KAERI_dataset/sample_submission.csv'
    sub = pd.read_csv(submission_csv_path)
    sub.iloc[:, 1:] = test_preds_all
    
    if do_wave:
        sub.to_csv("./wavenet_preds.csv", index=False)
    else:
        sub.to_csv("./res18_preds.csv", index=False)
        
    print('over')

In [ ]:
do_train(train, test, do_wave=True)
do_train(train, test, do_wave=False)

## 6. 결과 및 결언
## Conclusion & Discussion

In [ ]:
def post_pro(sub):
    sub.loc[sub.X < -400, 'X'] = -400
    sub.loc[sub.X > 400, 'X'] = 400

    sub.loc[sub.Y < -400, 'Y'] = -400
    sub.loc[sub.Y > 400, 'Y'] = 400

    sub.loc[sub.M < 25, 'M'] = 25
    sub.loc[sub.M > 175, 'M'] = 175

    sub.loc[sub.V < 0.2, 'V'] = 0.2
    sub.loc[sub.V > 1.0, 'V'] = 1.0
    return sub

In [ ]:
cnn_submit = pd.read_csv('./cnn_nf32_pred.csv')
cnn_flip_submit = pd.read_csv('./cnn_nf32_flip_pred.csv')

cnn_nf16_submit = pd.read_csv('./cnn_nf16_pred.csv')
cnn_nf16_flip_submit = pd.read_csv('./cnn_nf16_flip_pred.csv')

cnn_nf16_gap_submit = pd.read_csv('./cnn_nf16_gap_pred.csv')
cnn_nf16_gap_flip_submit = pd.read_csv('./cnn_nf16_gap_flip_pred.csv')

cnn_gap_submit = pd.read_csv('./cnn_nf32_gap_pred.csv')
cnn_gap_flip_submit = pd.read_csv('./cnn_nf32_gap_flip_pred.csv')

cnn_375_submit = pd.read_csv('./cnn_nf16_375_pred.csv')
cnn_gap_375_submit = pd.read_csv('./cnn_nf16_gap_375_pred.csv')

wavenet_submit = pd.read_csv('./wavenet_preds.csv')
res18_submit = pd.read_csv('./res18_preds.csv')

In [ ]:
cnn_submit = post_pro(cnn_submit)
cnn_flip_submit = post_pro(cnn_flip_submit)

cnn_nf16_submit = post_pro(cnn_nf16_submit)
cnn_nf16_flip_submit = post_pro(cnn_nf16_flip_submit)

cnn_nf16_gap_submit = post_pro(cnn_nf16_gap_submit)
cnn_nf16_gap_flip_submit = post_pro(cnn_nf16_gap_flip_submit)

cnn_gap_submit = post_pro(cnn_gap_submit)
cnn_gap_flip_submit = post_pro(cnn_gap_flip_submit)

cnn_375_submit = post_pro(cnn_375_submit)
cnn_gap_375_submit = post_pro(cnn_gap_375_submit)

wavenet_submit = post_pro(wavenet_submit)
res18_submit = post_pro(res18_submit)

In [ ]:
sub = pd.read_csv('../data/KAERI_dataset/sample_submission.csv')

In [ ]:
# X Y
X_Y_pred = (cnn_submit.iloc[:, 1:3].values + cnn_flip_submit.iloc[:, 1:3].values + cnn_nf16_submit.iloc[:, 1:3].values + 
       cnn_nf16_flip_submit.iloc[:, 1:3].values + cnn_nf16_gap_submit.iloc[:, 1:3].values + cnn_nf16_gap_flip_submit.iloc[:, 1:3].values + 
        cnn_gap_submit.iloc[:, 1:3].values + cnn_gap_flip_submit.iloc[:, 1:3].values + 
        cnn_375_submit.iloc[:, 1:3].values + cnn_gap_375_submit.iloc[:, 1:3].values + 
        res18_submit.iloc[:, 1:3].values + wavenet_submit.iloc[:, 1:3].values
       ) / 12

In [ ]:
# M V
M_V_pred = (cnn_submit.iloc[:, 3:].values + cnn_flip_submit.iloc[:, 3:].values + cnn_nf16_submit.iloc[:, 3:].values + 
       cnn_nf16_flip_submit.iloc[:, 3:].values + cnn_nf16_gap_submit.iloc[:, 3:].values + cnn_nf16_gap_flip_submit.iloc[:, 3:].values + 
        cnn_gap_submit.iloc[:, 3:].values + cnn_gap_flip_submit.iloc[:, 3:].values
       ) / 8
M_V_pred = M_V_pred * 0.5 + wavenet_submit.iloc[:, 3:].values * 0.2 + res18_submit.iloc[:, 3:].values * 0.3

In [ ]:
sub.iloc[:, 1:3] = X_Y_pred
sub.iloc[:, 3:] = M_V_pred

In [ ]:
sub.to_csv('./result_pred.csv',index=False)